# Call Center


Resource planning is the bread and butter of any sucessful business.

The following use-case is adopted from the [simmer mailing list](https://groups.google.com/g/simmer-devel/c/gsr6F7CJQf8/m/euW1ZaU0DAAJ)

Let's assume a customer service chat support, where the arrivals are requests from customers and the resources will have to respond to those messages. The process works as such: 

* The requests arrive throughout the day, but should join a pooled queue to wait for an available responder. 
* The responders are available in two shifts, excluding weekends; ideally the two shifts should not have individual queues, since there is already a pooled queue
* If a responder from Shift A is working on a ticket but is about to end their shift, they will turnover the message that their working on to Shift B.
* Shifts A and B will have different capacities 

Except for weekends, since there are no available shifts, so it will be worked on by the first available responder the following week.

## Shift System

In particular choosing a shift model & shift capacity planning are key for many sucessful business operations. Because of complex dynamics and interplay, it's often very hard to work out the capacity and bottlenecks in such systems on paper.

Let's try to model the process described above using kalasim. First, we load the library and import the core functions with

In [1]:
@file:Repository("*mavenLocal")

%useLatestDescriptors on
//%use kalasim(0.12-SNAPSHOT)
@file:DependsOn("com.github.holgerbrandl:kalasim:0.12-SNAPSHOT")



**Note**: Notebook execution is currently blocked by https://github.com/Kotlin/kotlin-jupyter/issues/355

Next, we work out the model
* Shifts are modelled as enums
* The shift system is modelled using a [sequence builder](https://kotlinlang.org/docs/sequences.html)
* Requests are modelled as `Component` to add the it to the request queue
* The buisness process is modelled as component as well
* All entities are bound into a simulation of type `CallCenter` for easy experimentation

In [2]:
import kotlin.math.roundToInt


enum class ShiftID { A, B, WeekEnd }

val shiftModel = sequence {
    while(true) {
        repeat(5) { yield(ShiftID.A); yield(ShiftID.B) }
        yield(ShiftID.WeekEnd)
    }
}


// model requests with static duration for now once they got hold of an operator
class Request : Component() {
    val callCenter = get<Resource>()

    override fun process() = sequence {
        request(callCenter, capacityLimitMode = CapacityLimitMode.SCHEDULE) {
            hold(1.minutes)
        }
    }
}

open class ShiftManager : Component() {
    val shiftIt = shiftModel.iterator()
    val callCenter = get<Resource>()

    override fun repeatedProcess() = sequence {
        val currentShift = shiftIt.next()

        log("starting new shift ${currentShift}")

        // adjust shift capacity at the beginning of the shift
        callCenter.capacity = when(currentShift) {
            ShiftID.A -> 2.0
            ShiftID.B -> 8.0
            ShiftID.WeekEnd -> 0.0
        }

        // wait for end of shift
        hold(if(currentShift == ShiftID.WeekEnd) 48.hours else 12.hours)
    }
}


abstract class CallCenter(val arrivalRate: Double = 0.3, logEvents: Boolean = true) : Environment(logEvents) {
    // not defined at this point
    abstract val shiftManager : ShiftManager

    val callCenter = dependency { Resource("Call Center") }

    init{
        ComponentGenerator(iat = exponential(arrivalRate)){ Request() }
    }
}


val sim =  object : CallCenter() {
    override val shiftManager = ShiftManager()
}

Line_4.jupyter.kts (47:99 - 108) Type mismatch: inferred type is Boolean but SimTime /* = Instant */ was expected
Line_4.jupyter.kts (54:9 - 27) This API is not recommended as the duration unit is not specified. Remove any doubt by adding .minutes, .hours, .days accordingly. The default duration unit of a simulation environment is MINUTES.

Let's run the unit for 1000 hours

In [ ]:
sim.run(600)

To understand the dynamics of the model we could now try inpspecting its progression. First we check out the queue length

In [ ]:
import org.kalasim.plot.kravis.display

sim.callCenter.requesters.queueLengthTimeline.display()


![](img.png)

## Shift Handover Precision

Clearly, this first version has the limitation, that tasks that overlap with a shift-change, do not immediately respect changes in capacity. That is, when changing from a shift with high to a shift with less capacity, ongoing tasks will be completed irrespective of the reduced capacity.

It's not straightforward to cancel these tasks to request them again in the next shift. This is because, a `release()` will - by design - check if new requests could be honored. So ongoing claims could be released easily, but requesting them again - even with higher priority - will cause them to be processed slightly after the immediatly honored request queue sucessors.

To solve the problem elegantly, we can use two other interactions namely `interrupt()` and `standby()`. With `interrupt()` we stop all ongoing tasks at a shift change. With `standby()` we can schedule process continauation in the next simulation cycle.

For the revised model we just need to model another `ShiftManager` with our revised hand-over process:

In [ ]:

class InterruptingShiftManager: ShiftManager() {
    override fun repeatedProcess() = sequence {
        val currentShift = shiftIt.next()

        log("starting new shift $currentShift")

        // adjust shift capacity at the beginning of the shift
        callCenter.capacity = when(currentShift) {
            ShiftID.A -> 2.0
            ShiftID.B -> 8.0
            ShiftID.WeekEnd -> 0.0
        }

        // complete hangover calls from previous shift
        fun shiftLegacy() = callCenter.claimers.components.filter { it.isInterrupted }

        // incrementally resume interrupted tasks while respecting new capacity
        while(shiftLegacy().isNotEmpty() && callCenter.capacity > 0) {
            val numRunning = callCenter.claimers.components.count { it.isScheduled }
            val spareCapacity = kotlin.math.max(0, callCenter.capacity.roundToInt() - numRunning)

            // resume interrupted tasks from last shift to max out new capacity
            shiftLegacy().take(spareCapacity).forEach { it.resume() }

            standby()
        }

        // wait for end of shift
        hold(if(currentShift == ShiftID.WeekEnd) 48.hours else 12.hours)

        // stop and reschedule the ongoing tasks
        callCenter.claimers.components.forEach {
            // detect remaining task time and request this with high prio so
            // that these tasks are picked up next in the upcoming shift
            it.interrupt()
        }
    }
}


We can now instantiate a new call center with the improved hand-over process

In [ ]:
val intSim = object: CallCenter() {
    override val shiftManager = InterruptingShiftManager()
}
 
intSim.run(600)

intSim.callCenter.requesters.queueLengthTimeline
    .display("Request queue length with revised handover process")


![](img_1.png)

Noteably, this model has the almost the same dynamics, but is more correct during shift handover.